In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('path to input data files'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import os
import shutil

# Paths for your two image folders and the new combined folder
folder1 = 'cataractdataset/dataset/1_normal'
folder2 = 'cataractdataset//dataset/2_cataract'
folder3 = 'cataractdataset//dataset/2_glaucoma'
folder4 = 'cataractdataset//dataset/3_retina_disease'
combined_folder = 'combined folder directory'

# Create the combined folder if it doesn't exist
if not os.path.exists(combined_folder):
    os.makedirs(combined_folder)

# Function to copy all files from a source to destination folder
def copy_files(source_folder, destination_folder):
    for filename in os.listdir(source_folder):
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)

        # Check if file already exists in destination folder
        if os.path.exists(destination_path):
            # If yes, rename the file to avoid overwrite
            basename, extension = os.path.splitext(filename)
            new_filename = basename + '_copy' + extension
            destination_path = os.path.join(destination_folder, new_filename)

        # Copy file to destination folder
        shutil.copy(source_path, destination_path)

# Copy images from both folders to the combined folder
copy_files(folder1, combined_folder)
copy_files(folder2, combined_folder)
copy_files(folder3, combined_folder)
copy_files(folder4, combined_folder)

print("Images from both folders have been combined.")
runtime_records = {}
metrics = {}

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import time

# Function to load and process images from the combined dataset directory
def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):  # Check if the item is a file
            img = load_img(img_path, target_size=(128, 128))  # Resize to 128x128
            img = img_to_array(img) / 255.0  # Normalize to [0, 1]
            images.append(img)
    return np.array(images)


# Load combined dataset
combined_dataset = load_images('combined folder directory')  # Update this path to your combined dataset directory

# Split dataset into training and testing
train_images, test_images = train_test_split(combined_dataset, test_size=0.2, random_state=42)

# Data preparation function
def build_data(image):
    lr = tf.image.resize(image, (64, 64))
    lr = tf.image.resize(lr, (128, 128), method=tf.image.ResizeMethod.BICUBIC)
    return (lr, image)

# Prepare training and testing datasets
train_data = tf.data.Dataset.from_tensor_slices(train_images).map(build_data).batch(32)
test_data = tf.data.Dataset.from_tensor_slices(test_images).map(build_data).batch(32)

# SRCNN Model
SRCNN_915 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, 9, padding='same', activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, 7, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(3, 5, padding='same')
])

# Compile the model
SRCNN_915.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

start_time = time.time()
# Train the model
SRCNN_915.fit(train_data, epochs=10, validation_data=test_data)
end_time = time.time()
runtime = end_time - start_time
runtime_records["SRCNN Original"] = runtime






In [ ]:
# Function to display test images and their super-resolved versio

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure  # For SSIM calculation
from skimage.metrics import structural_similarity as ssim

from scipy.ndimage import sobel  # For sharpness calculation

def display_test_images(test_data, model):
    for lr, hr in test_data.take(5):
        # Generate super-resolved (SR) image from low-resolution (LR) input
        sr = model.predict(lr[np.newaxis, ...])

        # Ensure lr, hr, and sr[0] are NumPy arrays
        lr_np = lr.numpy() if isinstance(lr, tf.Tensor) else lr
        hr_np = hr.numpy() if isinstance(hr, tf.Tensor) else hr
        sr_np = sr[0].numpy() if isinstance(sr[0], tf.Tensor) else sr[0]

        # Calculate PSNR values
        psnr_lr_hr = tf.image.psnr(lr, hr, max_val=1.0).numpy()
        psnr_sr_hr = tf.image.psnr(sr[0], hr, max_val=1.0).numpy()

        # Calculate SSIM values
        # Calculate SSIM values with a smaller window size
        ssim_lr_hr = ssim(lr_np, hr_np, data_range=hr_np.max() - hr_np.min(), multichannel=True, win_size=3)
        ssim_sr_hr = ssim(sr_np, hr_np, data_range=hr_np.max() - hr_np.min(), multichannel=True, win_size=3)

        # Calculate sharpness difference
        def calc_sharpness(image):
            gx = sobel(image, axis=0)
            gy = sobel(image, axis=1)
            return np.sqrt(gx**2 + gy**2).mean()

        sharpness_hr = calc_sharpness(hr_np)
        sharpness_sr = calc_sharpness(sr_np)
        sharpness_difference = sharpness_sr - sharpness_hr

        # Plot LR, SR, and HR images along with their metrics
        plt.figure(figsize=(18, 6))

        plt.subplot(1, 3, 1)
        plt.title(f'Low Resolution\nPSNR: {psnr_lr_hr:.2f} dB\nSSIM: {ssim_lr_hr:.2f}')
        plt.imshow(lr_np)
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title(f'Super Resolution\nPSNR: {psnr_sr_hr:.2f} dB\nSSIM: {ssim_sr_hr:.2f}\nSharpness Diff: {sharpness_difference:.2f}')
        plt.imshow(sr_np)
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title('Original Resolution')
        plt.imshow(hr_np)
        plt.axis('off')

        plt.show()
    return psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr

# Display some test images and their super-resolved versions along with the metrics
psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr = display_test_images(test_data.unbatch(), SRCNN_915)
metrics["SRCNN"] = [psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr]



In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Edge Loss Function
def edge_loss(y_true, y_pred):
    sobel_x = tf.constant([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=tf.float32)
    sobel_x_filter = tf.reshape(sobel_x, [3, 3, 1, 1])
    sobel_y_filter = tf.transpose(sobel_x_filter, [1, 0, 2, 3])

    # Apply filters to each channel separately
    def apply_filters(images):
        channels = tf.split(images, num_or_size_splits=3, axis=-1)  # Split the images into channels
        edge_images = []
        for channel in channels:
            edge_x = tf.nn.conv2d(channel, sobel_x_filter, strides=[1, 1, 1, 1], padding='SAME')
            edge_y = tf.nn.conv2d(channel, sobel_y_filter, strides=[1, 1, 1, 1], padding='SAME')
#             edge = tf.sqrt(tf.square(edge_x) + tf.square(edge_y))
            edge = tf.sqrt(tf.square(edge_x) + tf.square(edge_y) + 1e-12)  # Adding a small epsilon value for numerical stability

            edge_images.append(edge)
        return tf.concat(edge_images, axis=-1)  # Combine edge information from all channels

    edges_true = apply_filters(y_true)
    edges_pred = apply_filters(y_pred)

    return tf.reduce_mean(tf.abs(edges_pred - edges_true))


# Combined Loss Function
def combined_loss(y_true, y_pred):
    alpha = 0.5  # Adjust the weight of edge loss as needed
    return tf.keras.losses.mean_squared_error(y_true, y_pred) + alpha * edge_loss(y_true, y_pred)

# Function to load and process images
def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):
            img = load_img(img_path, target_size=(128, 128))
            img = img_to_array(img) / 255.0
            images.append(img)
    return np.array(images)

# Load dataset and split
combined_dataset = load_images('combined folder directory')
train_images, test_images = train_test_split(combined_dataset, test_size=0.2, random_state=42)

# Data preparation function
def build_data(image):
    lr = tf.image.resize(image, (64, 64))
    lr = tf.image.resize(lr, (128, 128), method=tf.image.ResizeMethod.BICUBIC)
    return (lr, image)

# Prepare datasets
train_data = tf.data.Dataset.from_tensor_slices(train_images).map(build_data).batch(32)
test_data = tf.data.Dataset.from_tensor_slices(test_images).map(build_data).batch(32)


SRCNN_915 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, 5, padding='same', activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),  # Additional layer for capturing more details
    tf.keras.layers.Conv2D(3, 3, padding='same')
])

# Compile the model with the combined loss
SRCNN_915.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy'])

start_time = time.time()
# Train the model
SRCNN_915.fit(train_data, epochs=10, validation_data=test_data)
end_time = time.time()
runtime = end_time - start_time
runtime_records["SRCNN_915"] = runtime
# Train the model


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure  # For SSIM calculation
from skimage.metrics import structural_similarity as ssim

from scipy.ndimage import sobel  # For sharpness calculation

def display_test_images(test_data, model):
    for lr, hr in test_data.take(5):
        # Generate super-resolved (SR) image from low-resolution (LR) input
        sr = model.predict(lr[np.newaxis, ...])

        # Ensure lr, hr, and sr[0] are NumPy arrays
        lr_np = lr.numpy() if isinstance(lr, tf.Tensor) else lr
        hr_np = hr.numpy() if isinstance(hr, tf.Tensor) else hr
        sr_np = sr[0].numpy() if isinstance(sr[0], tf.Tensor) else sr[0]

        # Calculate PSNR values
        psnr_lr_hr = tf.image.psnr(lr, hr, max_val=1.0).numpy()
        psnr_sr_hr = tf.image.psnr(sr[0], hr, max_val=1.0).numpy()

        # Calculate SSIM values
        # Calculate SSIM values with a smaller window size
        ssim_lr_hr = ssim(lr_np, hr_np, data_range=hr_np.max() - hr_np.min(), multichannel=True, win_size=3)
        ssim_sr_hr = ssim(sr_np, hr_np, data_range=hr_np.max() - hr_np.min(), multichannel=True, win_size=3)

        # Calculate sharpness difference
        def calc_sharpness(image):
            gx = sobel(image, axis=0)
            gy = sobel(image, axis=1)
            return np.sqrt(gx**2 + gy**2).mean()

        sharpness_hr = calc_sharpness(hr_np)
        sharpness_sr = calc_sharpness(sr_np)
        sharpness_difference = sharpness_sr - sharpness_hr

        # Plot LR, SR, and HR images along with their metrics
        plt.figure(figsize=(18, 6))

        plt.subplot(1, 3, 1)
        plt.title(f'Low Resolution\nPSNR: {psnr_lr_hr:.2f} dB\nSSIM: {ssim_lr_hr:.2f}')
        plt.imshow(lr_np)
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title(f'Super Resolution\nPSNR: {psnr_sr_hr:.2f} dB\nSSIM: {ssim_sr_hr:.2f}\nSharpness Diff: {sharpness_difference:.2f}')
        plt.imshow(sr_np)
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title('Original Resolution')
        plt.imshow(hr_np)
        plt.axis('off')

        plt.show()
    return psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr

# Display some test images and their super-resolved versions along with the metrics
psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr = display_test_images(test_data.unbatch(), SRCNN_915_modified)
metrics["SRCNN Edge Loss"] = [psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr]


In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Edge Loss Function
def edge_loss(y_true, y_pred):
    sobel_x = tf.constant([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=tf.float32)
    sobel_x_filter = tf.reshape(sobel_x, [3, 3, 1, 1])
    sobel_y_filter = tf.transpose(sobel_x_filter, [1, 0, 2, 3])

    # Apply filters to each channel separately
    def apply_filters(images):
        channels = tf.split(images, num_or_size_splits=3, axis=-1)
        edge_images = []
        for channel in channels:
            edge_x = tf.nn.conv2d(channel, sobel_x_filter, strides=[1, 1, 1, 1], padding='SAME')
            edge_y = tf.nn.conv2d(channel, sobel_y_filter, strides=[1, 1, 1, 1], padding='SAME')
            edge = tf.sqrt(tf.square(edge_x) + tf.square(edge_y) + 1e-12)
            edge_images.append(edge)
        return tf.concat(edge_images, axis=-1)

    edges_true = apply_filters(y_true)
    edges_pred = apply_filters(y_pred)

    return tf.reduce_mean(tf.abs(edges_pred - edges_true))

# Combined Loss Function
def combined_loss(y_true, y_pred):
    alpha = 0.5  # Adjust the weight of edge loss as needed
    return tf.keras.losses.mean_squared_error(y_true, y_pred) + alpha * edge_loss(y_true, y_pred)

# Custom Sharpening Layer
class SharpeningLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(SharpeningLayer, self).__init__()
        self.sharpening_kernel = tf.constant([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=tf.float32)
        self.sharpening_kernel = self.sharpening_kernel[:, :, tf.newaxis, tf.newaxis]
        self.sharpening_kernel = tf.tile(self.sharpening_kernel, [1, 1, 3, 1])

    def call(self, inputs):
        return tf.nn.depthwise_conv2d(inputs, self.sharpening_kernel, strides=[1, 1, 1, 1], padding='SAME')

# Function to load and process images
def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):
            img = load_img(img_path, target_size=(128, 128))
            img = img_to_array(img) / 255.0
            images.append(img)
    return np.array(images)

# Load dataset and split
combined_dataset = load_images('combined folder directory')  # Update the path as per your dataset location
train_images, test_images = train_test_split(combined_dataset, test_size=0.2, random_state=42)

# Data preparation function
def build_data(image):
    lr = tf.image.resize(image, (64, 64))
    lr = tf.image.resize(lr, (128, 128), method=tf.image.ResizeMethod.BICUBIC)
    return (lr, image)

# Prepare datasets
train_data = tf.data.Dataset.from_tensor_slices(train_images).map(build_data).batch(32)
test_data = tf.data.Dataset.from_tensor_slices(test_images).map(build_data).batch(32)

# Modified SRCNN Model with smaller kernel sizes and a sharpening layer at the end
SRCNN_sharpening = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, 5, padding='same', activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),  # Additional layer for capturing more details
    tf.keras.layers.Conv2D(3, 3, padding='same'),
    SharpeningLayer()  # Custom sharpening layer
])

# Compile the model with the combined loss
SRCNN_sharpening.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy'])

start_time = time.time()
# Train the model
SRCNN_sharpening.fit(train_data, epochs=10, validation_data=test_data)
end_time = time.time()
runtime = end_time - start_time
runtime_records["SRCNN Sharpening"] = runtime
# Train the model


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure  # For SSIM calculation
from skimage.metrics import structural_similarity as ssim

from scipy.ndimage import sobel  # For sharpness calculation

def display_test_images(test_data, model):
    for lr, hr in test_data.take(5):
        # Generate super-resolved (SR) image from low-resolution (LR) input
        sr = model.predict(lr[np.newaxis, ...])

        # Ensure lr, hr, and sr[0] are NumPy arrays
        lr_np = lr.numpy() if isinstance(lr, tf.Tensor) else lr
        hr_np = hr.numpy() if isinstance(hr, tf.Tensor) else hr
        sr_np = sr[0].numpy() if isinstance(sr[0], tf.Tensor) else sr[0]

        # Calculate PSNR values
        psnr_lr_hr = tf.image.psnr(lr, hr, max_val=1.0).numpy()
        psnr_sr_hr = tf.image.psnr(sr[0], hr, max_val=1.0).numpy()

        # Calculate SSIM values
        # Calculate SSIM values with a smaller window size
        ssim_lr_hr = ssim(lr_np, hr_np, data_range=hr_np.max() - hr_np.min(), multichannel=True, win_size=3)
        ssim_sr_hr = ssim(sr_np, hr_np, data_range=hr_np.max() - hr_np.min(), multichannel=True, win_size=3)

        # Calculate sharpness difference
        def calc_sharpness(image):
            gx = sobel(image, axis=0)
            gy = sobel(image, axis=1)
            return np.sqrt(gx**2 + gy**2).mean()

        sharpness_hr = calc_sharpness(hr_np)
        sharpness_sr = calc_sharpness(sr_np)
        sharpness_difference = sharpness_sr - sharpness_hr

        # Plot LR, SR, and HR images along with their metrics
        plt.figure(figsize=(18, 6))

        plt.subplot(1, 3, 1)
        plt.title(f'Low Resolution\nPSNR: {psnr_lr_hr:.2f} dB\nSSIM: {ssim_lr_hr:.2f}')
        plt.imshow(lr_np)
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title(f'Super Resolution\nPSNR: {psnr_sr_hr:.2f} dB\nSSIM: {ssim_sr_hr:.2f}\nSharpness Diff: {sharpness_difference:.2f}')
        plt.imshow(sr_np)
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title('Original Resolution')
        plt.imshow(hr_np)
        plt.axis('off')

        plt.show()
    return psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr

# Display some test images and their super-resolved versions along with the metrics
psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr = display_test_images(test_data.unbatch(), SRCNN_915_modified_with_sharpening)
metrics["SRCNN Sharpening"] = [psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr]


In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard


# Edge Loss Function
def edge_loss(y_true, y_pred):
    sobel_x = tf.constant([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=tf.float32)
    sobel_x_filter = tf.reshape(sobel_x, [3, 3, 1, 1])
    sobel_y_filter = tf.transpose(sobel_x_filter, [1, 0, 2, 3])

    def apply_filters(images):
        channels = tf.split(images, num_or_size_splits=3, axis=-1)
        edge_images = []
        for channel in channels:
            edge_x = tf.nn.conv2d(channel, sobel_x_filter, strides=[1, 1, 1, 1], padding='SAME')
            edge_y = tf.nn.conv2d(channel, sobel_y_filter, strides=[1, 1, 1, 1], padding='SAME')
            edge = tf.sqrt(tf.square(edge_x) + tf.square(edge_y) + 1e-12)
            edge_images.append(edge)
        return tf.concat(edge_images, axis=-1)

    edges_true = apply_filters(y_true)
    edges_pred = apply_filters(y_pred)

    return tf.reduce_mean(tf.abs(edges_pred - edges_true))

def combined_loss(y_true, y_pred):
    alpha = 0.5  # Adjust the weight of edge loss as needed
    return tf.keras.losses.mean_squared_error(y_true, y_pred) + alpha * edge_loss(y_true, y_pred)

class SharpeningLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(SharpeningLayer, self).__init__()
        self.sharpening_kernel = tf.constant([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=tf.float32)
        self.sharpening_kernel = self.sharpening_kernel[:, :, tf.newaxis, tf.newaxis]
        self.sharpening_kernel = tf.tile(self.sharpening_kernel, [1, 1, 3, 1])

    def call(self, inputs):
        return tf.nn.depthwise_conv2d(inputs, self.sharpening_kernel, strides=[1, 1, 1, 1], padding='SAME')

class ChannelAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, reduction_ratio=16):
        super(ChannelAttentionLayer, self).__init__()
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        self.dense1 = tf.keras.layers.Dense(input_shape[-1] // self.reduction_ratio, activation='relu', use_bias=False)
        self.dense2 = tf.keras.layers.Dense(input_shape[-1], activation='sigmoid', use_bias=False)

    def call(self, inputs):
        gap = tf.reduce_mean(inputs, axis=[1, 2], keepdims=True)
        channel_wise_interaction = self.dense1(gap)
        scale = self.dense2(channel_wise_interaction)
        return inputs * scale

def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):
            img = load_img(img_path, target_size=(128, 128))
            img = img_to_array(img) / 255.0
            images.append(img)
    return np.array(images)

combined_dataset = load_images('combined folder directory')  # Update the path as per your dataset location
train_images, test_images = train_test_split(combined_dataset, test_size=0.2, random_state=42)

def build_data(image):
    lr = tf.image.resize(image, (64, 64))
    lr = tf.image.resize(lr, (128, 128), method=tf.image.ResizeMethod.BICUBIC)
    return (lr, image)

train_data = tf.data.Dataset.from_tensor_slices(train_images).map(build_data).batch(32)
test_data = tf.data.Dataset.from_tensor_slices(test_images).map(build_data).batch(32)

# Modified SRCNN Model with Channel Attention and a sharpening layer at the end
SRCNN_attention = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, 5, padding='same', activation='relu', input_shape=(128, 128, 3)),
    ChannelAttentionLayer(),  # Channel Attention after the first convolutional layer
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    ChannelAttentionLayer(),  # Channel Attention after the second convolutional layer
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    ChannelAttentionLayer(),  # Channel Attention after the third convolutional layer
    tf.keras.layers.Conv2D(3, 3, padding='same'),
    SharpeningLayer()  # Custom sharpening layer
])

SRCNN_attention.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy'])
tensorboard_callback = TensorBoard(log_dir="./logs", histogram_freq=1, write_graph=True)
start_time = time.time()
# Train the model
SRCNN_attention.fit(train_data, epochs=50, validation_data=test_data)
end_time = time.time()
runtime = end_time - start_time
runtime_records["ESRCNN"] = runtime


In [ ]:
# Plot the model
from tensorflow.keras.utils import plot_model
# plot_path = 'model_architecture.png'
# plot_model(SRCNN_915_with_attention, to_file=plot_path, show_shapes=True, show_layer_names=True, expand_nested=True)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure  # For SSIM calculation
from skimage.metrics import structural_similarity as ssim

from scipy.ndimage import sobel  # For sharpness calculation

def display_test_images(test_data, model):
    for lr, hr in test_data.take(5):
        # Generate super-resolved (SR) image from low-resolution (LR) input
        sr = model.predict(lr[np.newaxis, ...])

        # Ensure lr, hr, and sr[0] are NumPy arrays
        lr_np = lr.numpy() if isinstance(lr, tf.Tensor) else lr
        hr_np = hr.numpy() if isinstance(hr, tf.Tensor) else hr
        sr_np = sr[0].numpy() if isinstance(sr[0], tf.Tensor) else sr[0]

        # Calculate PSNR values
        psnr_lr_hr = tf.image.psnr(lr, hr, max_val=1.0).numpy()
        psnr_sr_hr = tf.image.psnr(sr[0], hr, max_val=1.0).numpy()

        # Calculate SSIM values
        # Calculate SSIM values with a smaller window size
        ssim_lr_hr = ssim(lr_np, hr_np, data_range=hr_np.max() - hr_np.min(), multichannel=True, win_size=3)
        ssim_sr_hr = ssim(sr_np, hr_np, data_range=hr_np.max() - hr_np.min(), multichannel=True, win_size=3)

        # Calculate sharpness difference
        def calc_sharpness(image):
            gx = sobel(image, axis=0)
            gy = sobel(image, axis=1)
            return np.sqrt(gx**2 + gy**2).mean()

        sharpness_hr = calc_sharpness(hr_np)
        sharpness_sr = calc_sharpness(sr_np)
        sharpness_difference = sharpness_sr - sharpness_hr

        # Plot LR, SR, and HR images along with their metrics
        plt.figure(figsize=(18, 6))

        plt.subplot(1, 3, 1)
        plt.title(f'Low Resolution\nPSNR: {psnr_lr_hr:.2f} dB\nSSIM: {ssim_lr_hr:.2f}')
        plt.imshow(lr_np)
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title(f'Super Resolution\nPSNR: {psnr_sr_hr:.2f} dB\nSSIM: {ssim_sr_hr:.2f}\nSharpness Diff: {sharpness_difference:.2f}')
        plt.imshow(sr_np)
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title('Original Resolution')
        plt.imshow(hr_np)
        plt.axis('off')

        plt.show()
    return psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr

# Display some test images and their super-resolved versions along with the metrics
psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr = display_test_images(test_data.unbatch(), SRCNN_915_with_attention)
metrics["ESRCNN"] = [psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr]


In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import time

# Edge Loss Function
def edge_loss(y_true, y_pred):
    sobel_x = tf.constant([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=tf.float32)
    sobel_x_filter = tf.reshape(sobel_x, [3, 3, 1, 1])
    sobel_y_filter = tf.transpose(sobel_x_filter, [1, 0, 2, 3])
    def apply_filters(images):
        channels = tf.split(images, num_or_size_splits=3, axis=-1)
        edge_images = []
        for channel in channels:
            edge_x = tf.nn.conv2d(channel, sobel_x_filter, strides=[1, 1, 1, 1], padding='SAME')
            edge_y = tf.nn.conv2d(channel, sobel_y_filter, strides=[1, 1, 1, 1], padding='SAME')
            edge = tf.sqrt(tf.square(edge_x) + tf.square(edge_y) + 1e-12)
            edge_images.append(edge)
        return tf.concat(edge_images, axis=-1)
    edges_true = apply_filters(y_true)
    edges_pred = apply_filters(y_pred)
    return tf.reduce_mean(tf.abs(edges_pred - edges_true))

def combined_loss(y_true, y_pred):
    y_pred_resized = tf.image.resize(y_pred, (tf.shape(y_true)[1], tf.shape(y_true)[2]), method='bilinear')
    alpha = 0.9
    return tf.keras.losses.mean_squared_error(y_true, y_pred_resized) + alpha * edge_loss(y_true, y_pred_resized)

# Sharpening Layer
class SharpeningLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(SharpeningLayer, self).__init__()
        self.sharpening_kernel = tf.constant([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=tf.float32)
        self.sharpening_kernel = self.sharpening_kernel[:, :, tf.newaxis, tf.newaxis]

    def build(self, input_shape):
        # Ensure the kernel matches the number of input channels
        num_channels = input_shape[-1]
        self.sharpening_kernel = tf.tile(self.sharpening_kernel, [1, 1, num_channels, 1])

    def call(self, inputs):
        return tf.nn.depthwise_conv2d(inputs, self.sharpening_kernel, strides=[1, 1, 1, 1], padding='SAME')

# Channel Attention Layer
class ChannelAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, reduction_ratio=16):
        super(ChannelAttentionLayer, self).__init__()
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        self.dense1 = tf.keras.layers.Dense(input_shape[-1] // self.reduction_ratio, activation='relu', use_bias=False)
        self.dense2 = tf.keras.layers.Dense(input_shape[-1], activation='sigmoid', use_bias=False)

    def call(self, inputs):
        gap = tf.reduce_mean(inputs, axis=[1, 2], keepdims=True)
        channel_wise_interaction = self.dense1(gap)
        scale = self.dense2(channel_wise_interaction)
        return inputs * scale

# Spatial Attention Layer
class SpatialAttentionLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(SpatialAttentionLayer, self).__init__()

    def build(self, input_shape):
        # Define Conv2D layer here to ensure it's created only once
        self.conv = tf.keras.layers.Conv2D(1, kernel_size=7, padding='same', activation='sigmoid')

    def call(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)
        # Use the previously defined Conv2D layer
        filters = self.conv(concat)
        return inputs * filters


class SubpixelConv2D(tf.keras.layers.Layer):
    def __init__(self, scale=2, **kwargs):
        super(SubpixelConv2D, self).__init__(**kwargs)
        self.scale = scale  # Ensure scale is correctly set for upscaling

    def call(self, inputs):
        # This layer will only be used for upscaling with valid scale factors
        if self.scale > 1:
            return tf.nn.depth_to_space(inputs, self.scale)
        else:
            raise ValueError("Scale factor must be greater than 1 for SubpixelConv2D")

    def compute_output_shape(self, input_shape):
        if self.scale > 1:
            return (input_shape[0], input_shape[1] * self.scale, input_shape[2] * self.scale, input_shape[3] // (self.scale ** 2))
        return input_shape



def load_images(directory):
    images = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):
            img = load_img(img_path, target_size=(128, 128))
            img = img_to_array(img) / 255.0
            images.append(img)
    return np.array(images)

combined_dataset = load_images('combined folder directory')  # Update the path as per your dataset location
train_images, test_images = train_test_split(combined_dataset, test_size=0.2, random_state=42)

def build_data(image):
    lr = tf.image.resize(image, (64, 64))
    lr = tf.image.resize(lr, (128, 128), method=tf.image.ResizeMethod.BICUBIC)
    return (lr, image)

train_data = tf.data.Dataset.from_tensor_slices(train_images).map(build_data).batch(32)
test_data = tf.data.Dataset.from_tensor_slices(test_images).map(build_data).batch(32)


SRCNN_915_with_attention = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, 5, padding='same', activation='relu', input_shape=(128, 128, 3)),
    ChannelAttentionLayer(reduction_ratio=16),  # Channel attention after the first convolution
    SpatialAttentionLayer(),  # Spatial attention added here
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    SharpeningLayer(),  # Sharpening layer added here
    SubpixelConv2D(scale=2),  # Upscaling to 256x256
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    ChannelAttentionLayer(reduction_ratio=16),
    tf.keras.layers.Conv2D(3, 3, padding='same', activation='relu'),  # Final output with correct dimensions
    tf.keras.layers.Resizing(128, 128)  # Resize back to original dimensions
])




SRCNN_915_with_attention.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1, write_graph=True)
start_time = time.time()
# Train the model
SRCNN_915_with_attention.fit(train_data, epochs=50, validation_data=test_data, callbacks=[tensorboard_callback])
end_time = time.time()
runtime = end_time - start_time
print(f"Training runtime: {runtime} seconds")
runtime_records["EEASRCNN"] = runtime


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure  # For SSIM calculation
from skimage.metrics import structural_similarity as ssim

from scipy.ndimage import sobel  # For sharpness calculation

def display_test_images(test_data, model):
    for lr, hr in test_data.take(5):
        # Generate super-resolved (SR) image from low-resolution (LR) input
        sr = model.predict(lr[np.newaxis, ...])

        # Ensure lr, hr, and sr[0] are NumPy arrays
        lr_np = lr.numpy() if isinstance(lr, tf.Tensor) else lr
        hr_np = hr.numpy() if isinstance(hr, tf.Tensor) else hr
        sr_np = sr[0].numpy() if isinstance(sr[0], tf.Tensor) else sr[0]

        # Calculate PSNR values
        psnr_lr_hr = tf.image.psnr(lr, hr, max_val=1.0).numpy()
        psnr_sr_hr = tf.image.psnr(sr[0], hr, max_val=1.0).numpy()

        # Calculate SSIM values
        # Calculate SSIM values with a smaller window size
        ssim_lr_hr = ssim(lr_np, hr_np, data_range=hr_np.max() - hr_np.min(), multichannel=True, win_size=3)
        ssim_sr_hr = ssim(sr_np, hr_np, data_range=hr_np.max() - hr_np.min(), multichannel=True, win_size=3)

        # Calculate sharpness difference
        def calc_sharpness(image):
            gx = sobel(image, axis=0)
            gy = sobel(image, axis=1)
            return np.sqrt(gx**2 + gy**2).mean()

        sharpness_hr = calc_sharpness(hr_np)
        sharpness_sr = calc_sharpness(sr_np)
        sharpness_difference = sharpness_sr - sharpness_hr

        # Plot LR, SR, and HR images along with their metrics
        plt.figure(figsize=(18, 6))

        plt.subplot(1, 3, 1)
        plt.title(f'Low Resolution\nPSNR: {psnr_lr_hr:.2f} dB\nSSIM: {ssim_lr_hr:.2f}')
        plt.imshow(lr_np)
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title(f'Super Resolution\nPSNR: {psnr_sr_hr:.2f} dB\nSSIM: {ssim_sr_hr:.2f}\nSharpness Diff: {sharpness_difference:.2f}')
        plt.imshow(sr_np)
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title('Original Resolution')
        plt.imshow(hr_np)
        plt.axis('off')

        plt.show()
    return psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr

# Display some test images and their super-resolved versions along with the metrics
psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr = display_test_images(test_data.unbatch(), SRCNN_915_with_attention)
metrics["EEASRCNN"] = [psnr_lr_hr, psnr_sr_hr, ssim_lr_hr, ssim_sr_hr, sharpness_hr, sharpness_sr]


In [ ]:
# Plots
import matplotlib.pyplot as plt


# Extracting names and runtime values
names = list(runtime_records.keys())
values = list(runtime_records.values())

# Creating the bar plot
plt.figure(figsize=(10, 6))  # You can adjust the figure size as needed
plt.bar(names, values, color='skyblue')  # You can change the color

plt.xlabel('Algorithm')  # X-axis label
plt.ylabel('Runtime (ms)')  # Y-axis label
plt.title('Runtime Measures of Algorithms')  # Plot title
plt.xticks(rotation=45)  # Rotate names for better readability if necessary
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Adding a grid for better readability; customize as needed

plt.show()



In [ ]:
import pandas as pd

# Define the column names for your metrics
columns = ['PSNR LR-HR', 'PSNR SR-HR', 'SSIM LR-HR', 'SSIM SR-HR', 'Sharpness HR', 'Sharpness SR']

# Create a DataFrame
df = pd.DataFrame.from_dict(metrics, orient='index', columns=columns)

# Reset index to add the model names as a separate column
df.reset_index(inplace=True)
df.rename(columns={'index': 'Model'}, inplace=True)

# Display the DataFrame
df.head(6)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Metrics names for labeling the plot
metric_names = ['PSNR LR-HR', 'PSNR SR-HR', 'SSIM LR-HR', 'SSIM SR-HR', 'Sharpness HR', 'Sharpness SR']

# Number of models and metrics
num_models = len(metrics)
num_metrics = len(metric_names)

# Create a 2D array from the metrics dictionary
data = np.array([metrics[model] for model in metrics])

# Set the positions and width for the bars
positions = np.arange(num_models)
bar_width = 0.1  # Adjust as necessary for clarity

# Create the plot
plt.figure(figsize=(12, 8))

for i in range(num_metrics):
    plt.bar(positions + i * bar_width, data[:, i], width=bar_width, label=metric_names[i])

# Adding labels and title
plt.xlabel('Model')
plt.ylabel('Metric Values')
plt.title('Comparison of Different Metrics Across Models')
plt.xticks(positions + bar_width * (num_metrics - 1) / 2, metrics.keys())
plt.legend(title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
plt.show()
